# "Computing Koselleck" Dashboard

* Please be patient as this page loads. It may take up to a few minutes. Once it loads you'll see a few graphs and tables about the word "culture".
* You can type in another word and hit "Analyze" to see another word.
* You can also **scroll to the bottom of the page to see a list of the most changing words**. Click on any of those words to see its data.
* Feel free to edit the annotation data on the top right panel. Words with annotation data and marked as valid have a <span style="color:lightblue">blue</span> instead of gray button at the bottom of the page; words marked as invalid (due to OCR errors, etc) have a <span style="color:lightcoral">red</span> background.

In [3]:
from ipynb.fs.full.koselleck import *
print('Bitte warten Sie einen Augenblick...',timer=False)

[Koselleck] (08:06:40) Bitte warten Sie einen Augenblick... 


In [4]:
# markdwn?

In [3]:
WORD='culture'

In [4]:
# list of top words
dfanno=read_sheet()
dfanno.rank_word=dfanno.rank_word.apply(int)
dfanno['rank']=dfanno['rank'].apply(int)
dfanno=dfanno.sort_values('rank_word')
# dfanno

In [5]:
all_words_by_nov = list(get_all_novelty_scores().groupby('word').mean().reset_index().sort_values(
    'foote_novelty_z',ascending=False).drop_duplicates('word').word)
all_words_by_lnm_rank = [w for w in dfanno.index]# if w in set(all_words_by_nov) and dfanno.loc[w]['rank']>0]
okwords=set(dfanno.index)|set(all_words_by_lnm_rank)
all_words_by_nov = [w for w in all_words_by_nov if w in okwords and dfanno.loc[w]['rank']>0]
# dfanno

In [6]:
# 'effective' in all_words_by_lnm_rank

In [7]:
# 'effective' in all_words_by_nov

In [2]:
signif_words=set(get_signif_novelty_words(p_peak=0.05))
keywords=get_keywords()
abswords=get_words_ever_abs()

NameError: name 'get_signif_novelty_words' is not defined

In [1]:
# interact_word=interact.options(manual=True, manual_name="Show data for word")

def show(word='slavery',num_proc=2):
    print('Generating plots',timer=False)
    fig_nbrs=plot_nbrs(word,width=8,height=5,ybin=20,year_min=1720,year_max=1900,max_rank1=10,max_rank=10,min_periods=1)
    print('Plotting distance matrix')
    fig_distmat=plot_historical_semantic_distance_matrix(word)
    print('Plotting novelty scores')
    fig_nov=plot_novelty_words(word,xlim0=1720,xlim1=1900)
    print('Saving temporary files')
    with tempfile.TemporaryDirectory() as tmpdirname:
        fig1fnfn=os.path.join(tmpdirname,'fig1.png')
        fig2fnfn=os.path.join(tmpdirname,'fig2.png')
        fig3fnfn=os.path.join(tmpdirname,'fig3.png')
        fig_nbrs.save(fig1fnfn)
        fig_distmat.save(fig2fnfn)
        fig_nov.save(fig3fnfn)
        
#         print('Creating layout')
        with open(fig1fnfn,'rb') as fig1b, open(fig2fnfn,'rb') as fig2b, open(fig3fnfn,'rb') as fig3b:
            img1=widgets.Image(value=fig1b.read(),format='png')
            img2=widgets.Image(value=fig2b.read(),format='png')
            img3=widgets.Image(value=fig3b.read(),format='png')
#             vb=VBox([img1,img3,img2])
            vl=[img1,img3,img2]
            print('Returning')
            return vl

In [10]:



def do_save(b=None):
    global dfanno
    print('Reloading metadata before writing',timer=False)
    dfanno1=dfanno=read_sheet()
    
    with output:
#         dfanno1=read_sheet()
        w=widg_word.value
        dfanno1.loc[w,'Changes']=widg_changes.value
        dfanno1.loc[w,'From']=widg_from.value
        dfanno1.loc[w,'To']=widg_to.value
        dfanno1.loc[w,'Notes']=widg_notes.value
        dfanno1.loc[w,'is_valid']=widg_valid.value
        for b in allbuttons:
            if b.description.split()[-1]==w:
                b.style.button_color='lightcoral' if widg_valid.value=='n' else 'lightblue'
            
#         display(dfanno1.loc[w])
#         display(dfanno1)
        dfanno1.rank_word=dfanno1.rank_word.apply(int)
        dfanno1=dfanno1.sort_values('rank_word')
        print('Saving online')
        write_sheet(dfanno1)
        print('Finished')
    
    
    

def is_anno(w):
    try:
        row=dfanno.loc[w]
    except KeyError:
        return False
    anno=False
    for k,v in row.items():
        if k[0] == k[0].upper() or k=='is_valid':
            if v.strip():
                anno=True
                break
    return anno

def is_valid(w):
    try:
        row=dfanno.loc[w]
    except KeyError:
        return False
    # assumes yes
    return row.is_valid!='n'

def get_color(w,color=None):
    if is_anno(w): color='lightblue'
    if not is_valid(w): color='lightcoral'
    return color


def do_anno_save(w,key,value):
    dfanno.loc[w,key]=value
    with output:
        print(f'Saved: {dfanno.loc[w,key]}')


In [29]:
# @output.capture(clear_output=True)
def submit_word(w):
    with output:
        widg_word.value=w
#         word_title.value=markdown.markdown('## '+w)
        output.clear_output()
        printm('----')
        printm('## '+w)
        try:
            widg_changes.value=dfanno.loc[w,'Changes']
            widg_from.value=dfanno.loc[w,'From']
            widg_to.value=dfanno.loc[w,'To']
            widg_notes.value=dfanno.loc[w,'Notes']
            widg_valid.value=dfanno.loc[w,'is_valid']
        except KeyError:
            widg_changes.value=''
            widg_from.value=''
            widg_to.value=''
            widg_notes.value=''
            widg_valid.value=''

        try:
            dfpsgs=show_sents_word(w,num_proc=4,ybin=50,n=5)
            for col in dfpsgs.columns: dfpsgs[col]=dfpsgs[col].apply(highlight_psg)
            dfpsgs_md=dfpsgs.reset_index().drop('index',1).to_markdown()
            img_nbr,img_nov,img_dist=show(w)
            wordinfodf=info_word_comparisons(w)
            
            stzdf=get_nbr_simple_sattelzeit(w).reset_index().drop('word',1)
            rank,perc=np.nan,np.nan
            if len(stzdf):
                stzrank,stzperc=stzdf.iloc[0]['Change Rank'],stzdf.iloc[0]['Change Percentile']
                stzdf=stzdf.drop(['Change Rank', 'Change Percentile'],1)
            stzdf_md=stzdf.to_markdown()
            
            tilts_stz_md=format_tilts_sattelzeit_html(w)
            
#         except AssertionError as e:
        except Exception as e:
            print(f'ERROR: Somehow there is insufficient data for this word. Please choose another.\n\n{e}')
            if widg_valid.value!='n':
                widg_valid.value='n'
                do_save()
            for b in allbuttons:
                if b.description.split()[-1]==w:
                    b.style.button_color='lightcoral'
            display(VBox([
                markdwn('## '+w),
                markdwn('<span style="color:lightcoral">ERROR: Somehow there is insufficient data for this word. Please choose another.</span>'),
                HBox([input_box,annobox])
            ]))
            return
    #         display(show(w),output)
    #         display(info_word_comparisons(w),output2)
    #         display(Markdown(mpsgs),output2)

        hb1=HBox([
            VBox([
#                 markdwn('### Input'),
#                 input_box,
                markdwn('### Top 10 closest words over time'),
                img_nbr,
                markdwn('### Historical semantic distance matrix'),
                img_dist,
                markdwn('### Foote Novelty plot'),
                img_nov
            ],layout=Layout(width='50%')),
            VBox([
                markdwn('### Annotations'),
                annobox,
                markdwn(f'### Meta neighborhood changes\n* Change Rank: {int(stzrank):,}\n* Change Percentile: {int(round(stzperc,0))}%'),
                markdwn(stzdf_md),
                markdwn('### Top vectors of change'),
                markdwn(tilts_stz_md),
            ],layout=Layout(width='50%'))
        ])
        display(hb1)    
    
        
        printm('### Passages')
        printm(dfpsgs_md)
        
#         for btn in allbuttons:
#             wx=btn.description.split()[-1]
# #             print(wx,get_color(wx))
#             btn.style.button_color=get_color(wx)
        
        

In [30]:
def make_button(i,w):
    return widgets.Button(
        description=f'{i+1}. {w}',
        layout=Layout(width='125px', height='25px'),
        style=ButtonStyle(button_color=get_color(w))
#     ) for i,w in enumerate(dfanno.index[:200])
    )
def make_buttons(l):
    return [make_button(i,w) for i,(_,w) in enumerate(l)]


In [31]:
# define widgets
widg_word=widgets.Text(value='culture',layout=Layout(width='auto'))
word=widg_word.value
widg_word_submit = widgets.Button(description='Analyze',layout=Layout(width='125px'))
output = widgets.Output()
widg_changes = widgets.Text(value='',description='Changes')
widg_from = widgets.Text(value='',description='From')
widg_to = widgets.Text(value='',description='To')
widg_notes = widgets.Text(value='',description='Notes')
widg_valid = widgets.Text(value='',description='is_valid')
savebutton = Button(description='Save annotations')
# set layout
input_box=HBox([widg_word,widg_word_submit])
display(input_box)
display(output)

Output()

In [32]:

# define behaviors
def on_widg_word_submit(b): submit_word(b.value)
def on_widg_word_button_submit(b): submit_word(widg_word.value)
def on_widg_word_button_submit_desc(b): submit_word(b.description.split()[-1])
# set behaviors
widg_word.on_submit(on_widg_word_submit)
widg_word_submit.on_click(on_widg_word_button_submit)
savebutton.on_click(do_save)


In [33]:

lim=100

# remove invalids?
# words_invalid = set(dfanno.query('is_valid=="n"').index)
all_words_by_lnm_rank=[x for x in all_words_by_lnm_rank if is_valid(x)]
all_words_by_nov=[x for x in all_words_by_nov if is_valid(x)]

buttons_nov=make_buttons([(i,w) for i,w in enumerate(all_words_by_nov) if w in signif_words][:lim])
buttons=make_buttons([(i,w) for i,w in enumerate(all_words_by_lnm_rank)][:lim])
buttons_kw=make_buttons([(i,w) for i,w in enumerate(all_words_by_lnm_rank) if w in keywords][:lim])
buttons_abs=make_buttons([(i,w) for i,w in enumerate(all_words_by_lnm_rank) if w in abswords][:lim])
buttons_nov_and_ch=make_buttons([(i,w) for i,w in enumerate(all_words_by_lnm_rank) if w in signif_words][:lim])


## dirs of change





allbuttons = buttons + buttons_nov + buttons_kw + buttons_abs + buttons_nov_and_ch
buttond={}

lim_dir=50

okvecs={'+Abs','+Conc','+Ambig','-Ambig','+Collective','+Indiv','+Human','+Object','+Judg','+Perc','+Pos','+Neg','+Polit','+Acad','+Qual','+Quant','+Sing','+Plural','+Time','+Space','+Woman','+Man'}
for k,l in tqdm(get_top_changes_stz().items(),desc='Building widgets',position=0,disable=True):
    if not k in okvecs: continue
    l=[x for x in l if is_valid(x)][:lim_dir]
    buttond[k]=make_buttons(list(enumerate(l)))
    allbuttons+=buttond[k]
#     break

for b in allbuttons:
    b.on_click(on_widg_word_button_submit_desc)


In [34]:
annobox=VBox([widg_changes,widg_from,widg_to,widg_notes,widg_valid,savebutton])

# hb_top=HBox([
#     VBox([
#         markdwn('#### Input'),
#         input_box
#     ]),
    
# ],layout=Layout(width='100%'))

# vbanno=VBox([
#         markdwn('#### Annotations'),
#         annobox
#     ])
# word_title=markdwn('## '+WORD)
# # display(word_title)
# # display(hb_top)

In [35]:

#         display(wboxes,output)
i=1
i2=1
i3=5
dirchangebox_l = []
for k,l in get_top_changes_stz().items():
    if not k in buttond: continue
    dirchangebox_l.append(
        VBox(
            [markdwn(f'#### Top {lim_dir} most {k} words')] + \
            [HBox(buttond[k][n:n+i]) for n in range(0,len(buttond[k])+i+1,i)],
            layout=Layout(border='1px dotted gray',width='125px')
        ),
    )
dirchangebox=VBox([markdwn('### Directions of change')] + [
    HBox(dirchangebox_l[n:n+i3])
    for n in range(0,len(dirchangebox_l)+i3+1,i3)
])
# dirchangebox


In [36]:
i=1
wid='125px'
magnchangebox=HBox([
    VBox(
        [markdwn(f'#### Top {lim} most<br/>changing among novel words (p<0.05)')] + \
        [HBox(buttons_nov_and_ch[n:n+i]) for n in range(0,len(buttons_nov_and_ch)+i+1,i)],
        layout=Layout(border='1px dotted gray',width=wid)
    ),
    VBox(
        [markdwn(f'#### Top {lim} most changing words')] + \
        [HBox(buttons[n:n+i]) for n in range(0,len(buttons)+i+1,i)],
        layout=Layout(border='1px dotted gray',width=wid)
    ),
    VBox(
        [markdwn(f'#### Top {lim} most "novel" words (p<0.05)')] + \
        [HBox(buttons_nov[n:n+i]) for n in range(0,len(buttons_nov)+i+1,i)],
        layout=Layout(border='1px dotted gray',width=wid)
    ),
    VBox(
        [markdwn(f'#### Top {lim} most changing keywords')] + \
        [HBox(buttons_kw[n:n+i2]) for n in range(0,len(buttons_kw)+i2+1,i2)],
        layout=Layout(border='1px dotted gray',width=wid)
    ),
    VBox(
        [markdwn(f'#### Top {lim} most changing abstractions')] + \
        [HBox(buttons_abs[n:n+i2]) for n in range(0,len(buttons_abs)+i2+1,i2)],
        layout=Layout(border='1px dotted gray',width=wid)
    )
])
magnchangebox=VBox([markdwn('### Magnitude of change'), magnchangebox])
# magnchangebox

In [37]:



hbox_buttons_words=HBox([
    magnchangebox,
    dirchangebox,
])

# word_buttons

In [38]:

display(hbox_buttons_words)

In [28]:
submit_word('culture')